In [13]:
import pandas as pd
import json
import networkx
import glob
import matplotlib.pyplot as plt

In [14]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5)

In [20]:
hotel_df_s = pd.read_csv("hotel_sens.csv")
hotel_df_s.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,sentiment
0,0,1160184816060514305,chefjuanboada,Día 2 en Hotel Castellmar https://www.instagra...,2019-08-10 13:43:09+00:00,0,0,NaN,Atacames,0.802902
1,1,1159839684081491969,HotelCARLUZ,Llego el fin de semana y te proponemos algunas...,2019-08-09 14:51:44+00:00,0,0,NaN,Atacames,0.688678
2,2,1157821700341239808,oscarmera_1,#amigos procesos de formación #friends #esmera...,2019-08-04 01:12:59+00:00,3,1,#amigos #friends #esmeraldas #feriado2019 #ins...,Atacames,0.149371
3,3,1157462434388566018,HotelElCisne1y2,Nuestros Queridos Turistas Felices De Conocer ...,2019-08-03 01:25:23+00:00,1,0,NaN,Atacames,0.990453
4,4,1157361204681224195,HotelCARLUZ,!!! Simplemente !!! pic.twitter.com/LH21VSlJyT,2019-08-02 18:43:08+00:00,0,0,NaN,Atacames,0.629843


In [21]:
hospedaje_df_s = pd.read_csv("hospedaje_sens.csv")
hospedaje_df_s.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,sentiment
0,0,1160185098916040704,carturoleon,Te faltó decir viajar 6 8 12 horas por tierra ...,2019-08-10 13:44:17+00:00,1,0,NaN,Gualaquiza,0.879095
1,1,1143526470935109632,riverpeopleraft,Ven y disfruta de conocer la Amazonía con Rive...,2019-06-25 14:28:51+00:00,0,0,NaN,Tena,0.151790
2,2,1159506853354758144,HeneciaEcuador,Buen día Henecias! Si alguien que viaja a Perú...,2019-08-08 16:49:11+00:00,14,5,NaN,El Chaco,0.722887
3,3,1159446378160017411,KattyMejia11,@Airbnb necesito asesoramiento para mecanismo ...,2019-08-08 12:48:52+00:00,0,0,NaN,El Chaco,0.571598
4,4,1159168036756230144,ABRELATAM,En estos últimos días los donantes que se han ...,2019-08-07 18:22:50+00:00,0,0,NaN,El Chaco,0.886135


In [22]:
airbnb_df_s = pd.read_csv("airbnb_sens.csv")
airbnb_df_s.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,sentiment
0,0,1160150293818089472,florencia3g,El tema del reportaje era hablar de la gentrif...,2019-08-10 11:25:59+00:00,5,0,NaN,San Cristobal,0.630286
1,1,1160047591926591488,Lakatus,¿Has probado Airbnb? Regístrate a través de mi...,2019-08-10 04:37:53+00:00,0,0,NaN,San Cristobal,0.934657
2,2,1159936574781452288,elsuperidolo,@SantanderMx buenas tardes si hago un pago con...,2019-08-09 21:16:44+00:00,0,0,NaN,San Cristobal,0.518644
3,3,1159936130180980736,elsuperidolo,Entonces si se considera como compra en el ext...,2019-08-09 21:14:58+00:00,0,0,NaN,San Cristobal,0.428252
4,4,1159934098749583360,elsuperidolo,@airbnb_mx una duda al hacer el pago del Airbn...,2019-08-09 21:06:54+00:00,0,0,NaN,San Cristobal,0.560564


# Analysis of Co-occurrence and Networks of Words
- In this part is going to be analyzed co-occuring words and the relations between terms is going to be showed
- source: https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/calculate-tweet-word-bigrams-networks-in-python/

In [52]:
import nltk
from nltk import bigrams
import itertools
import collections

In [53]:
tweet_hotel = pd.unique(hotel_df_s['2']).tolist()
tweet_hospedaje = pd.unique(hospedaje_df_s['2']).tolist()
tweet_airbnb = pd.unique(airbnb_df_s['2']).tolist()

In [54]:
tweet_hotel

['Día 2 en Hotel Castellmar https://www.instagram.com/p/B0_E1F_pABbHlUjpclYzhGVmx5g67YA3uptlCE0/?igshid=2fikksbf8irn …',
 'Llego el fin de semana y te proponemos algunas alternativas para tu diversión . Tomar el sol  disfrutar de nuestras instalaciones o dar un tranquilo paseo al atardecer en la playa . ¡Te esperamos!pic.twitter.com/ahJKvUGtll',
 '#amigos procesos de formación #friends #esmeraldas #feriado2019 #instalike #likey #instagram #loveyourself #pride #lullaby #igot7 #bestmoment #oppa #fotodeldia #txt #tomorrowxtogether en Hotel Perla Verde -… https://www.instagram.com/p/B0uSNdNpcb3/?igshid=1rlqjm8cewud3 …',
 'Nuestros Queridos Turistas Felices De Conocer Desde El Aire Nuestra Playa Gracias Parapente Esmeraldaspic.twitter.com/HXAFPqYxYb',
 '!!! Simplemente !!! pic.twitter.com/LH21VSlJyT',
 'Como estar en casita #ElEquipo de grandpacifichotelecuador #comodidad #BuenaAtención #Tonsupa #Paz #Bolón no se puede pedir más #Síganlos en Grand Pacific Hotel https://www.instagram.com/p/B

In [55]:
terms_hotel = [list(bigrams(tweet)) for tweet in tweet_hotel]
terms_hospedaje = [list(bigrams(tweet)) for tweet in tweet_hospedaje]
terms_airbnb = [list(bigrams(tweet)) for tweet in tweet_airbnb]

In [56]:
terms_hotel[0]

[('D', 'í'),
 ('í', 'a'),
 ('a', ' '),
 (' ', '2'),
 ('2', ' '),
 (' ', 'e'),
 ('e', 'n'),
 ('n', ' '),
 (' ', 'H'),
 ('H', 'o'),
 ('o', 't'),
 ('t', 'e'),
 ('e', 'l'),
 ('l', ' '),
 (' ', 'C'),
 ('C', 'a'),
 ('a', 's'),
 ('s', 't'),
 ('t', 'e'),
 ('e', 'l'),
 ('l', 'l'),
 ('l', 'm'),
 ('m', 'a'),
 ('a', 'r'),
 ('r', ' '),
 (' ', 'h'),
 ('h', 't'),
 ('t', 't'),
 ('t', 'p'),
 ('p', 's'),
 ('s', ':'),
 (':', '/'),
 ('/', '/'),
 ('/', 'w'),
 ('w', 'w'),
 ('w', 'w'),
 ('w', '.'),
 ('.', 'i'),
 ('i', 'n'),
 ('n', 's'),
 ('s', 't'),
 ('t', 'a'),
 ('a', 'g'),
 ('g', 'r'),
 ('r', 'a'),
 ('a', 'm'),
 ('m', '.'),
 ('.', 'c'),
 ('c', 'o'),
 ('o', 'm'),
 ('m', '/'),
 ('/', 'p'),
 ('p', '/'),
 ('/', 'B'),
 ('B', '0'),
 ('0', '_'),
 ('_', 'E'),
 ('E', '1'),
 ('1', 'F'),
 ('F', '_'),
 ('_', 'p'),
 ('p', 'A'),
 ('A', 'B'),
 ('B', 'b'),
 ('b', 'H'),
 ('H', 'l'),
 ('l', 'U'),
 ('U', 'j'),
 ('j', 'p'),
 ('p', 'c'),
 ('c', 'l'),
 ('l', 'Y'),
 ('Y', 'z'),
 ('z', 'h'),
 ('h', 'G'),
 ('G', 'V'),
 ('V', 'm'),

In [57]:
bigrams_hotel = list(itertools.chain(*terms_hotel))
bigrams_hospedaje = list(itertools.chain(*terms_hospedaje))
bigrams_airbnb = list(itertools.chain(*terms_airbnb))

In [73]:
counts_hotel_bg = collections.Counter(bigrams_hotel)
counts_hospedaje_bg = collections.Counter(bigrams_hospedaje)
counts_airbnb_bg = collections.Counter(bigrams_airbnb)

In [74]:
#bigram_hotel = counts_hotel_bg.most_common(20)
#palabras = []
#num_coincid = []
#for x in bigram_hotel:
    #palabras.append(x[0])
    #num_coincid.append(x[1])
#hotel_df_s['Bigrams']=Palabras

In [75]:
bgm_hotel = pd.DataFrame(counts_hotel_bg.most_common(20),
                             columns=['bigram', 'count'])

bgm_hotel.head()

,bigram,count
0,"(a, )",23179
1,"(e, )",19495
2,"(o, )",17588
3,"(s, )",16912
4,"(n, )",16066


In [79]:
bgm_hospedaje = pd.DataFrame(counts_hospedaje_bg.most_common(20),
                             columns=['bigram', 'count'])

bgm_hospedaje.head()

,bigram,count
0,"(e, )",1598
1,"(a, )",1426
2,"(s, )",1419
3,"(o, )",1120
4,"(o, s)",1054


In [80]:
bgm_airbnb = pd.DataFrame(counts_airbnb_bg.most_common(20),
                             columns=['bigram', 'count'])

bgm_airbnb.head()

,bigram,count
0,"(e, )",1053
1,"(a, )",976
2,"(s, )",906
3,"(o, )",777
4,"(e, n)",763
